# 🚀 Install, Import, and Log in

### Step 0️⃣: Install W&B

In [ ]:
%%capture
#!pip install wandb
!pip install pandas

### Step 1️⃣: Import W&B and Login

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from tensorflow.keras.callbacks import ReduceLROnPlateau


#%matplotlib inline

In [ ]:
import wandb
from wandb.keras import WandbCallback

#wandb.init()  # defaults are over-ridden during the sweep


#wandb.login()

In [ ]:

fluxData_df =pd.read_hdf("data/fluxData_df.h5", key="fluxData_df")

zernikeData_df =pd.read_hdf("data/zernikeData_df.h5", key="zernikeData_df")

Read data in

# 👩‍🍳 Prepare Dataset

In [ ]:
# Prepare the training dataset
x_train = fluxData_df
y_train = zernikeData_df

x_train_norm = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0)
input_train = x_train_norm.to_numpy()[:,:,np.newaxis]

# 🧠 Define the wandb sweep

In [ ]:
os.environ["WANDB_NOTEBOOK_NAME"] = "AoConv1DKerasStudio.ipynb"

sweep_config = {
    "method": "random", # try grid or random
    "metric": {
        "name": "val_root_mean_squared_error",
        "goal": "minimize"
    },
    "parameters": {

    "learning_rate" :{
        "values": [ 0.0005, 0.001]
        }, 
    "lrFactor": {
        "values": [0.5]
    },                  
  
    "batch_size": {
        "values": [128]
    },
    "epochs": {
        "values": [100]
    }, 
    "conv1D_1": {
        "values": [64, 128]
    }, 
    "conv1D_1_width": {
        "values": [4, 8]
    }, 
    "conv1D_2": {
        "values": [32, 64]
    }, 
    "conv1D_2_width": {
        "values": [2, 4]
    }                
    }
}

In [ ]:
## Define training loop

In [ ]:
def train():
 
  wandb.init(tags=["Conv1D"])  # defaults are over-ridden during the sweep
  config = wandb.config
  input_shape = (4, 19, 1)

  AoConv1DModel = keras.Sequential([
      keras.layers.Conv1D( config.conv1D_1, config.conv1D_1_width, activation='relu',input_shape=input_shape[1:]),
      keras.layers.MaxPool1D(pool_size=2),
      keras.layers.Conv1D( config.conv1D_2, config.conv1D_2_width, activation='relu',input_shape=input_shape[1:]),
      keras.layers.MaxPool1D(pool_size=2),
      keras.layers.Flatten(),
      keras.layers.Dense(2000, activation="relu"),
      keras.layers.Dense(1050, activation="relu"),
      keras.layers.Dense(100, activation="relu"),
      keras.layers.Dense(9,  name="predictions"),

  ])


  earlyStopping = keras.callbacks.EarlyStopping(monitor = "val_loss", patience =10)
  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=config.lrFactor,
                                patience=5, min_lr=0.000001, verbose=1, cooldown= 5)

  opt = keras.optimizers.Adam(learning_rate=config.learning_rate)
  AoConv1DModel.compile(loss= keras.losses.MeanSquaredError(),  optimizer=opt, metrics= [tf.keras.metrics.RootMeanSquaredError()])
  AoConv1DModel.fit(input_train, y_train, batch_size=config.batch_size, epochs=config.epochs,validation_split = 0.2, verbose = 2, callbacks=[reduce_lr, earlyStopping, WandbCallback()])



## Initialize sweep

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity = "uwe-sterr",project="KerasAoSweep")
wandb.run

## Run sweep

In [ ]:
wandb.agent(sweep_id, train, count=400)